In [17]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import shapiro
from statsmodels.formula.api import ols


In [18]:
CURRENT_DIR = Path.cwd()

In [19]:
filepath = os.path.join(CURRENT_DIR, "S03_residual_stress_merge.xlsx")
dfm = pd.read_excel(filepath).rename(
    columns={"Sigma(x)": "sigma_x_pre", "FWHM": "FWHM_pre"}
)
dfm

,section,sample_no,location,R,W,D,sigma_x_post,FWHM_post,sigma_x_pre,FWHM_pre,diff_sigma_x
0,AA5052,1,1,1400,60,10,13,2.55,-15,2.50,28
1,AA5052,2,1,1400,60,15,16,2.51,2,2.47,14
2,AA5052,3,1,1400,60,20,19,2.47,9,2.48,10
3,AA5052,4,1,1400,70,10,20,2.45,10,2.48,10
4,AA5052,5,1,1400,70,15,6,2.47,0,2.49,6
...,...,...,...,...,...,...,...,...,...,...,...
1142,Center,50,7,1600,70,15,2,2.45,0,0.00,2
1143,Center,51,7,1600,70,20,2,2.45,0,0.00,2
1144,Center,52,7,1600,80,10,5,2.54,0,0.00,5
1145,Center,53,7,1600,80,15,1,2.41,0,0.00,1


In [20]:
# Standardize columnes R, W, D

for col in ["R", "W", "D"]:
    mean = dfm[col].mean()
    std = dfm[col].std()
    dfm[col] = (dfm[col] - mean) / std
dfm

,section,sample_no,location,R,W,D,sigma_x_post,FWHM_post,sigma_x_pre,FWHM_pre,diff_sigma_x
0,AA5052,1,1,-1.234711,-1.212550,-1.222132,13,2.55,-15,2.50,28
1,AA5052,2,1,-1.234711,-1.212550,0.001494,16,2.51,2,2.47,14
2,AA5052,3,1,-1.234711,-1.212550,1.225119,19,2.47,9,2.48,10
3,AA5052,4,1,-1.234711,0.010664,-1.222132,20,2.45,10,2.48,10
4,AA5052,5,1,-1.234711,0.010664,0.001494,6,2.47,0,2.49,6
...,...,...,...,...,...,...,...,...,...,...,...
1142,Center,50,7,1.207036,0.010664,0.001494,2,2.45,0,0.00,2
1143,Center,51,7,1.207036,0.010664,1.225119,2,2.45,0,0.00,2
1144,Center,52,7,1.207036,1.233879,-1.222132,5,2.54,0,0.00,5
1145,Center,53,7,1.207036,1.233879,0.001494,1,2.41,0,0.00,1


## Understanding the ANOVA table columns:

- `df`: Degrees of freedom - the number of independent pieces of information for each source
- `sum_sq`: Sum of squares - the total variation attributed to each factor
- `mean_sq`: Mean square - sum of squares divided by degrees of freedom (variance estimate)
- `F`: F-statistic - the ratio of factor variance to residual variance
- `PR(>F)`: P-value - probability of seeing this F-statistic by chance alone

## What is Type

- typ=1 (Type I): sequential SS, each term tested in the order it appears in the formula.
- typ=2, each term (factor or covariate) is tested after all other main effects but ignoring higher‑order interactions that include it.
  This is usually recommended when the model is reasonably balanced and you want tests that respect the marginality principle (main effects evaluated in the presence of other main effects, but not “penalized” by interactions).
- typ=3 (Type III): each term tested after all other terms including interactions; often used in software like SPSS, especially for unbalanced designs, but interpretation of main effects with strong interactions can be tricky.


In [21]:
# Filter the DataFrame for specific sections
filt = (dfm["section"].isin(["AA5052", "AA6061", "Center"])) 
dfa = dfm[filt]

# Define the formula for the model
formula = "diff_sigma_x ~ R + W + D + section"

# Fit the model
model = ols(formula, data=dfa).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model)

# Print the ANOVA results
anova_results.sort_values(by="PR(>F)")

,df,sum_sq,mean_sq,F,PR(>F)
section,2.0,3.313076e+05,165653.818514,134.715679,2.993434e-53
W,1.0,6.618948e+03,6618.947889,5.382768,2.051231e-02
R,1.0,1.542714e+03,1542.713638,1.254591,2.629132e-01
D,1.0,7.491263e+02,749.126263,0.609217,4.352448e-01
Residual,1141.0,1.403036e+06,1229.655073,NaN,NaN


In [22]:
# Filter the DataFrame for specific sections
filt = dfm["section"].isin(["AA5052"]) 
# filt = dfm["section"].isin(["AA5052"]) & (dfm["location"] == 1)
dfa = dfm[filt]

# Define the formula for the model
base = "diff_sigma_x ~ "
t_main = ["R", "W", "D"]
t_interact_2 = ["R:W", "R:D", "W:D"]
t_interact_3 = ["R:W:D"]
t_single = ["I(R)", "I(W)", "I(D)"]
t_quad = ["I(R**2)", "I(W**2)", "I(D**2)"]
t_quad_pair = ["I(R*W)", "I(R*D)", "I(W*D)"]
t_triple_1 = [
    "I(R**2*W)",
    "I(R**2*D)",
    "I(W**2*R)",
    "I(W**2*D)",
    "I(D**2*R)",
    "I(D**2*W)",
]
t_triple_all = ["I(R*W*D)"]

formula = base + " + ".join(
    t_main
    + t_interact_2
    + t_interact_3
    + t_single
    + t_quad
    + t_quad_pair
    + t_triple_1
    + t_triple_all
)
print(formula)

# Fit the model
model = ols(formula, data=dfa).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model)

# Print the ANOVA results
anova_results.sort_values(by="PR(>F)")
display(anova_results)

coefficients = model.params
print(coefficients)

diff_sigma_x ~ R + W + D + R:W + R:D + W:D + R:W:D + I(R) + I(W) + I(D) + I(R**2) + I(W**2) + I(D**2) + I(R*W) + I(R*D) + I(W*D) + I(R**2*W) + I(R**2*D) + I(W**2*R) + I(W**2*D) + I(D**2*R) + I(D**2*W) + I(R*W*D)


,df,sum_sq,mean_sq,F,PR(>F)
R,1.0,2056.196254,2056.196254,7.539180,0.006333
W,1.0,136.493750,136.493750,0.500463,0.479743
D,1.0,556.355844,556.355844,2.039916,0.154064
R:W,1.0,343.517391,343.517391,1.259529,0.262470
R:D,1.0,137.602673,137.602673,0.504529,0.477965
W:D,1.0,1148.800336,1148.800336,4.212153,0.040840
R:W:D,1.0,0.119795,0.119795,0.000439,0.983291
I(R),1.0,31.275773,31.275773,0.114675,0.735076
I(W),1.0,179.405656,179.405656,0.657803,0.417860
I(D),1.0,119.218133,119.218133,0.437121,0.508928


Intercept        21.772227
R                 1.342875
W                -0.054147
D                 0.996119
R:W              -0.454937
R:D              -0.345585
W:D              -0.855687
R:W:D             0.002302
I(R)              1.342875
I(W)             -0.054147
I(D)              0.996119
I(R ** 2)        -1.588769
I(W ** 2)         1.278981
I(D ** 2)         2.511988
I(R * W)         -0.454937
I(R * D)         -0.345585
I(W * D)         -0.855687
I(R ** 2 * W)     0.723033
I(R ** 2 * D)    -2.160604
I(W ** 2 * R)    -0.978340
I(W ** 2 * D)    -1.059149
I(D ** 2 * R)     0.562405
I(D ** 2 * W)    -0.036535
I(R * W * D)      0.002302
dtype: float64


In [23]:
# Filter the DataFrame for specific sections
filt = dfm["section"].isin(["AA6061"])
dfa = dfm[filt]

# Define the formula for the model
base = "diff_sigma_x ~ "
t_main = ["R", "W", "D"]
t_interact_2 = ["R:W", "R:D", "W:D"]
t_interact_3 = ["R:W:D"]
t_quad = ["I(R**2)", "I(W**2)", "I(D**2)"]
t_quad_pair = ["I(R*W)", "I(R*D)", "I(W*D)"]
t_triple_1 = [
    "I(R**2*W)",
    "I(R**2*D)",
    "I(W**2*R)",
    "I(W**2*D)",
    "I(D**2*R)",
    "I(D**2*W)",
]
t_triple_all = ["I(R*W*D)"]

formula = base + " + ".join(
    t_main
    + t_interact_2
    + t_interact_3
    + t_quad
    + t_quad_pair
    + t_triple_1
    + t_triple_all
)
print(formula)

# Fit the model
model = ols(formula, data=dfa).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model)

# Print the ANOVA results
anova_results.sort_values(by="PR(>F)")

diff_sigma_x ~ R + W + D + R:W + R:D + W:D + R:W:D + I(R**2) + I(W**2) + I(D**2) + I(R*W) + I(R*D) + I(W*D) + I(R**2*W) + I(R**2*D) + I(W**2*R) + I(W**2*D) + I(D**2*R) + I(D**2*W) + I(R*W*D)


,df,sum_sq,mean_sq,F,PR(>F)
W,1.0,1.251834e+04,12518.344035,3.771626,0.052896
R:W:D,1.0,1.154445e+04,11544.454469,3.478205,0.062982
I(R ** 2 * W),1.0,8.327680e+03,8327.680005,2.509029,0.114059
W:D,1.0,7.354298e+03,7354.298099,2.215761,0.137469
I(D ** 2 * R),1.0,6.961619e+03,6961.618916,2.097452,0.148401
I(W * D),1.0,5.871103e+03,5871.102673,1.768892,0.184347
D,1.0,3.979050e+03,3979.049582,1.198840,0.274274
I(W ** 2),1.0,3.682120e+03,3682.119578,1.109378,0.292912
I(D ** 2),1.0,3.680339e+03,3680.338663,1.108842,0.293029
I(W ** 2 * R),1.0,2.773039e+03,2773.038816,0.835483,0.361294


In [24]:
# Filter the DataFrame for specific sections
filt = dfm["section"].isin(["Center"])
dfa = dfm[filt]

# Define the formula for the model
base = "diff_sigma_x ~ "
t_main = ["R", "W", "D"]
t_interact_2 = ["R:W", "R:D", "W:D"]
t_interact_3 = ["R:W:D"]
t_quad = ["I(R**2)", "I(W**2)", "I(D**2)"]
t_quad_pair = ["I(R*W)", "I(R*D)", "I(W*D)"]
t_triple_1 = [
    "I(R**2*W)",
    "I(R**2*D)",
    "I(W**2*R)",
    "I(W**2*D)",
    "I(D**2*R)",
    "I(D**2*W)",
]
t_triple_all = ["I(R*W*D)"]

formula = base + " + ".join(
    t_main
    + t_interact_2
    + t_interact_3
    + t_quad
    + t_quad_pair
    + t_triple_1
    + t_triple_all
)
print(formula)

# Fit the model
model = ols(formula, data=dfa).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model)

# Print the ANOVA results
anova_results.sort_values(by="PR(>F)")

diff_sigma_x ~ R + W + D + R:W + R:D + W:D + R:W:D + I(R**2) + I(W**2) + I(D**2) + I(R*W) + I(R*D) + I(W*D) + I(R**2*W) + I(R**2*D) + I(W**2*R) + I(W**2*D) + I(D**2*R) + I(D**2*W) + I(R*W*D)


,df,sum_sq,mean_sq,F,PR(>F)
R,1.0,525.777778,525.777778,10.580321,0.001251
W,1.0,325.037706,325.037706,6.540793,0.010951
I(R ** 2),1.0,198.059874,198.059874,3.985595,0.046642
I(R ** 2 * W),1.0,151.474951,151.474951,3.048158,0.081679
R:W:D,1.0,137.595771,137.595771,2.768865,0.096982
I(R * D),1.0,119.457482,119.457482,2.403865,0.121912
D,1.0,69.168907,69.168907,1.391898,0.238862
W:D,1.0,59.917561,59.917561,1.205732,0.272911
R:W,1.0,55.929535,55.929535,1.125480,0.289451
I(R ** 2 * D),1.0,51.823760,51.823760,1.042859,0.307841
